In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
df=pd.read_csv("final_ds_data.csv").drop(columns="Unnamed: 0")

In [5]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,salary_in_inr,abroad
0,2020,MI,FT,Data Scientist,70000.0,EUR,79833,DE,0,Germany,L,6359496.78,1
1,2020,MI,FT,Data Engineering Manager,51999.0,EUR,59303,DE,100,Germany,S,4724076.98,1
2,2020,EN,PT,ML Engineer,14000.0,EUR,15966,DE,100,Germany,S,1271851.56,1
3,2020,EN,FT,Data Engineer,48000.0,EUR,54742,PK,100,Germany,L,4360747.72,1
4,2020,SE,FT,Principal Data Scientist,130000.0,EUR,148261,DE,100,Germany,M,11810471.26,1


In [85]:
currencies=list(df['salary_currency'].unique())

In [86]:
currencies

['EUR',
 'USD',
 'JPY',
 'GBP',
 'HUF',
 'INR',
 'CNY',
 'MXN',
 'CAD',
 'DKK',
 'PLN',
 'SGD',
 'BRL',
 'CLP',
 'CHF',
 'TRY',
 'AUD']

In [88]:
#using api to convert to real time currency value
def currency_convert(currencies,source):
    values=[]
    for x in currencies:
        url = "https://api.apilayer.com/currency_data/convert?to="+source+"&from="+x+"&amount=1"

        payload = {}
        headers= {
          "apikey": "pcELBns9GqsT0vr3nV7EcdTWsLB7RYGY"
        }

        response = requests.request("GET", url, headers=headers, data = payload)

        status_code = response.status_code
        result = response.text
        values.append(round(float(result.replace("\n","").replace(" ","").replace('}','').split(",")[-1].split(":")[-1]),2))
    return values

In [89]:
values_usd=currency_convert(currencies,"USD")

In [90]:
values_usd

[1.02,
 1.0,
 0.01,
 1.17,
 0.0,
 0.01,
 0.14,
 0.05,
 0.77,
 0.14,
 0.22,
 0.72,
 0.19,
 0.0,
 1.05,
 0.05,
 0.69]

In [91]:
values_inr=currency_convert(currencies,"INR")

In [92]:
values_inr

[80.93,
 79.57,
 0.56,
 92.97,
 0.2,
 1.0,
 11.49,
 4.01,
 61.26,
 10.89,
 17.21,
 56.98,
 15.46,
 0.09,
 83.3,
 4.36,
 54.76]

In [107]:
conv_df=pd.DataFrame({'Currencies':currencies,'USD_Rate per currency value':values_usd,'INR_Rate per currency value':values_inr})

In [110]:
conv_df.set_index("Currencies",inplace=True)

In [111]:
conv_df

,USD_Rate per currency value,INR_Rate per currency value
Currencies,,
EUR,1.02,80.93
USD,1.00,79.57
JPY,0.01,0.56
GBP,1.17,92.97
HUF,0.00,0.20
INR,0.01,1.00
CNY,0.14,11.49
MXN,0.05,4.01
CAD,0.77,61.26


In [121]:
def conv_salary_usd(row):
    index=row[0]
    val=row[1]
    rate=conv_df.loc[index][0]
    return val*rate

In [125]:
def conv_salary_inr(row):
    index=row[0]
    val=row[1]
    rate=conv_df.loc[index][1]
    return round(val*rate,2)

In [122]:
df['salary_in_usd']=df[['salary_currency','salary']].apply(conv_salary_usd,axis='columns')

In [126]:
df['salary_in_inr']=df[['salary_currency','salary']].apply(conv_salary_inr,axis='columns')

# EDA

Comparison of salaries for same roles available in India and other Countries

In [136]:
roles_ind=df.loc[df.abroad==0,'job_title'].unique()

In [137]:
roles_ind

array(['Data Scientist', 'Data Science Manager',
       'Data Science Consultant', 'Product Data Analyst', 'Data Analyst',
       '3D Computer Vision Researcher', 'Lead Data Analyst',
       'Data Engineer', 'Lead Data Scientist', 'Big Data Engineer',
       'Machine Learning Engineer', 'Head of Machine Learning',
       'Business Data Analyst', 'AI Scientist'], dtype=object)

In [157]:
df1=df[df.job_title.isin(roles_ind)]

In [164]:
#create a pivot table
sal_comp=pd.pivot_table(df1,index='job_title',columns='abroad',values='salary_in_inr',aggfunc=max,fill_value=0)

We can see that majority of the roles offered abroad pay a higher scale when compared in terms of INR

In [168]:
sal_comp.sort_values(by=[1,0],ascending=False)

abroad,0,1
job_title,,
Data Scientist,3224716.46,32782840
Data Engineer,2423894.48,25780680
Machine Learning Engineer,5278669.90,19892500
Data Science Manager,7541013.90,19176370
AI Scientist,1438101.98,15914000
Data Analyst,483695.52,15914000
Lead Data Scientist,3231806.20,15118300
Lead Data Analyst,1562052.94,13526900
Business Data Analyst,1469089.72,10741950


In [171]:
sal_comp=sal_comp.reset_index()

In [182]:
#lets plot a bargraph to illustrate this
fig=px.bar(sal_comp,x='job_title',y=[0,1],title="Overall Max Salary Comparison India and Abroad in terms of INR")
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [187]:
#check the same for Hybrid, WFH and On-Site in India
#Remote Ratio : 0,50,100, 100:WFH, 50: Hybrid,0:onsite
sal_comp2=pd.pivot_table(df1[df1.abroad==0],index='job_title',columns='remote_ratio',values='salary_in_inr',aggfunc=max,fill_value=0)

In [188]:
sal_comp2

remote_ratio,0,50,100
job_title,,,
3D Computer Vision Researcher,0.00,430880.94,0.00
AI Scientist,0.00,0.00,1438101.98
Big Data Engineer,1801192.26,0.00,1292722.48
Business Data Analyst,0.00,0.00,1469089.72
Data Analyst,483695.52,0.00,0.00
Data Engineer,0.00,1723603.42,2423894.48
Data Science Consultant,0.00,454619.62,0.00
Data Science Manager,0.00,7541013.90,0.00
Data Scientist,3224716.46,2369964.66,2518450.90


In [190]:
#lets plot a stacked bargraph to illustrate this
sal_comp2=sal_comp2.reset_index()
fig=px.bar(sal_comp2,x='job_title',y=[0,50,100],title="Overall Max Salary Comparison in India on Remote Ratio Basis")
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

<ul>
    <li> Data Science Manager Role in India offers the highest pay for Hybrid opportunities </li>
    <li> Machine Learning Engineer has the highest pay on-site</li>
    <li> Data Scientist Role has the highest WFH pay</li>

In [193]:
sal_comp3=pd.pivot_table(df1[df1.abroad==0],index='job_title',columns='experience_level',values='salary_in_inr',aggfunc=max)

In [194]:
sal_comp3

experience_level,EN,EX,MI,SE
job_title,,,,
3D Computer Vision Researcher,NaN,NaN,430880.94,NaN
AI Scientist,1438101.98,NaN,NaN,NaN
Big Data Engineer,1292722.48,NaN,1801192.26,NaN
Business Data Analyst,NaN,NaN,1469089.72,NaN
Data Analyst,483695.52,NaN,NaN,NaN
Data Engineer,2423894.48,NaN,NaN,NaN
Data Science Consultant,454619.62,NaN,NaN,NaN
Data Science Manager,NaN,NaN,NaN,7541013.9
Data Scientist,2369964.66,NaN,3224716.46,NaN


In [198]:
sal_comp3=sal_comp3.reset_index()
fig=px.bar(sal_comp3,x='job_title',y=['EN','MI','SE','EX'],title="Overall Max Salary Comparison in India on Experience Level")
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

<ul>
    <li> Head of Machine Learning role is only available at Expert level having a high payscale </li>
    <li> Data Engineer, AI Scientist, Data Analyst, Data Science Consultant have only entry level roles with Data Engineer having the highest pay scale </li>
    <li> Data Science Manager has the highest payscale at Senior level </li>
    <li> Data Scientist Role has the highest mid senior level payscale </li>

<strong>Because of the necessity of vast domain expertise and skills, Data Scientists and Engineers are relatively paid higher than Data Analysts

In [ ]:
#let's see which country offers the highest number of roles
#let's use an api to get the coordinates of the company location
// Forward Geocoding API Endpoint

http://api.positionstack.com/v1/forward
    ? access_key = YOUR_ACCESS_KEY
    & query = 1600 Pennsylvania Ave NW, Washington DC
    
// optional parameters: 

    & limit = 10
    & output = json

In [ ]:
url = "http://api.positionstack.com/v1/forward
    ? access_key = YOUR_ACCESS_KEY
    & query = 1600 Pennsylvania Ave NW, Washington DC"

        payload = {}
        headers= {
          "apikey": "pcELBns9GqsT0vr3nV7EcdTWsLB7RYGY"
        }

        response = requests.request("GET", url, headers=headers, data = payload)
